<a href="https://colab.research.google.com/github/ilnegozio/labs/blob/main/labs/lab05_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 5
----------------
#### Name(s): Denis
#### Date: February 5th

By the end of this lab you should be able to:
  * apply both a direct (ie. brute force) and heuristic algorithm to a problem
  * use the networkx module to visualize graphs
  * measure algorithm scaling


During this lab, you will be introduced to the following:
  * seeding the random module



## The Traveling Salesperson Problem

This lab is based on the famous Traveling Salesperson Problem (TSP) outlined in Section 3.3.2 of 'On Complexity'.  Imagine you needed to travel to ten different cities.  What is the sortest possible route you could take to visit every city?  For our version of the problem, you start from a specific city and must end on the same city.

In this lab, you will implement a direct (ie. brute force) and a heuristic solution to the problem.

In [ ]:
# start with some imports
import random, math, time, itertools
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

## Set up the map

For this implementation of the problem we will randomly place N cities on a 100 by 100 grid.  Cities will be labelled using a single character; 'A', 'B', 'C', etc.  City locations will be denoted by a 2-tuple, `(x,y)`, giving their position on the grid. In order to create reproducable maps, we will use [random.seed()](https://docs.python.org/3/library/random.html#random.seed) to initialize the random module.  

In [ ]:
def setup_map(N, seed=1):
  ''' Returns a map of N cities with random locations on a 100 by 100 grid '''
  random.seed(seed)
  city_locations = [(random.randint(0, 100), random.randint(0, 100)) for i in range(N)]
  city_names = [chr(65 + i) for i in range(N)]
  city_map = dict(zip(city_names, city_locations))
  return city_map

def graph_map(city_map):
  ''' graph the map of cities using networkx'''
  G = nx.Graph()
  G.add_nodes_from(city_map)
  nx.draw(G, city_map, with_labels=True)

def distance(city1, city2, city_map):
  ''' Returns the distance between two cities on a map'''
  x1, y1 = city_map[city1]
  x2, y2 = city_map[city2]
  return round(math.sqrt((x2 - x1)**2 + (y2 - y1)**2),1)

def calculate_distances(city_map):
  ''' Returns a dictionary with the distances between all pairs of cities in a map '''
  distances = {}
  for city1 in city_map:
      for city2 in city_map:
          if city1 != city2:
              # this will have duplicate distances (A,B) and (B,A)
              # these duplicates make the algorithm implememntation simpler
              distances[(city1, city2)] = distance(city1, city2, city_map)
  return distances


In [ ]:
# test our setup functions
city_map = setup_map(10)
print(f'City Map: {city_map}')
distances = calculate_distances(city_map)
print(f'Distances between cities: {distances}')
graph_map(city_map)

### Exercise 1: The Brute Force Solution

Implement a brute force solution to the traveling salesman problem using the city map and distance dictionary given above.  The algorithm for the brute force solution looks something like this:
 1. Loop over every possible route that starts and ends on city A.  For a three city example, this would be ABCA, ACBA.  
 2. Calculate the total distance of each possible route.  
 3. Find the route with the minimum total distance.

Before you start coding, think about what functions and/or class you might want to simplify your implementation of the algorithm.  Write a small test for your algorithm.

In [ ]:
# Ex. 1 your solution here

def permutation(elements):
  if len(elements) == 0:
    return [[]]
  result = []
  for i in range(len(elements)):
    rest = elements[:i] + elements[i+1:] # remove one element
    for perm in permutation(rest):
      result.append([elements[i]] + perm)
  return result

def total_distance(route, distances):
  dist = 0
  for i in range(len(route) - 1):
    dist += distances.get((route[i], route[i + 1]), 99999)
  return dist

def brute_force(city_map, distances):
  min_distance = float('inf') # no route assumption
  best_route = None
  for perm in permutation(list(city_map.keys())[1:]):
    route = ('A') + tuple(perm) + ('A')
    dist = total_distance(route, distances)
    if dist < min_distance:
      min_distance = dist
      best_route = route # update best option

  return best_route, min_distance


### Exercise 2: Nearest Neighbour

Implement a nearest neighbour solution to the traveling salesman problem using the city map and distance dictionary given above.  The algorithm for the nearest neighbour solution looks something like this:
 1. Start from the starting city.  
 2. Go to the nearest city that has not been visited.  
 3. Repeat steps 1 and 2 until all cities have been visited.

Before you start coding, think about what functions and/or class you might want to simplify your implementation of the algorithm.  Write a small test for your algorithm.

In [ ]:
# Ex.2 your code here

def nearest_neighbour(city_map, distances):
  cities = list(city_map.keys())
  unvisited = set(cities)
  unvisited.remove('A')
  route = ['A']
  total_dist = 0

  current_city = 'A'
  while unvisited:
    nearest_city = None
    nearest_dist = float('inf')
    for city in unvisited:
      if distances.get((current_city, city), 99999) < nearest_dist:
        nearest_dist = distances[(current_city, city)]
        nearest_city = city
    if nearest_city:
      total_dist += nearest_dist
      route.append(nearest_city) # append the closest
      unvisited.remove(nearest_city)
      current_city = nearest_city

  route.append('A') # return back
  total_dist += distances.get((current_city, 'A'), 99999)

  return route, total_dist

### Exercise 3: Graph the Routes

Use the networkx module to create directed graphs of the routes (brute force and nearest neighbour).  Include the distances between the cities on the edges.  Run several different map setups with a different sizes (ie. number of cities).  Do the brute force (ie. direct) and nearest neighbour (ie. heuristic) algorithms always give the same solution?  Do you notice anything about the brute force solutions that suggests an alternative heuristic algorithm?


In [ ]:
# Ex.3 your code here

def plot(city_map, route, title):
  for i in range(len(route) - 1):
    city1, city2 = route[i], route[i + 1]
    nx.DiGraph().add_edge(city1, city2, weight=distances.get((city1, city2), 99999))

  plt.figure(figsize=(8, 6))
  nx.draw(nx.DiGraph(), city_map, with_labels=True, node_color='red', edge_color='black', node_size=500, font_size=10)
  labels = nx.get_edge_attributes(nx.DiGraph(), 'weight')
  nx.draw_networkx_edge_labels(nx.DiGraph(), city_map, edge_labels=labels)
  plt.title(title)
  plt.show()

### Exercise 4: Algorithm Scaling

Use the `time` module to measure the running time of the two algorithms.  Note that the `time.time()` function returns the current time, so calling it before and after the algorithm has run can give you the time it takes for the algorithm to execute.  Taking the average over several trials will make the measurement more precise.  Demonstrate the scaling nature of the algorithms by plotting their execution time as a function of N, up to about N = 10.  

In [ ]:
# Ex 4: your code here

brute_list = []
neighbour_list = []

for i in range(10):
  t1 = time.time()
  plot(city_map, best_route, "brute force")
  t2 = time.time()
  t3 = t2 - t1
  brute_list.append(t3)
  print("brute force execution time:", t3)
  t1 = time.time()
  plot(city_map, nn_route, "nearest neighbour")
  t2 = time.time()
  t3 = t2 - t1
  neighbour_list.append(t3)
  print("nearest neighbour execution time:", t3)

brute_average = sum(brute_list) / len(brute_list)
print("brute force average execution time:", brute_average)
neighbour_average = sum(neighbour_list) / len(neighbour_list)
print("neighbour average execution time:", neighbour_average)